In [15]:
import Augmentor
from PIL import Image
from tqdm import tqdm
import os
import re

image_dir = "./images"
p = Augmentor.Pipeline(image_dir)
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth("./SegmentationClassAug")

p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
p.flip_left_right(probability=1)
p.flip_top_bottom(probability=1)
p.skew(probability=1)
p.random_distortion(probability=1, grid_width=6, grid_height=6, magnitude=8)
p.sample(1000)


Executing Pipeline:   0%|          | 0/1000 [00:00<?, ? Samples/s]

Initialised with 10 image(s) found.
Output directory set to ./images/output.9 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGBA size=1416x312 at 0x7F001C924DA0>: 100%|██████████| 1000/1000 [00:58<00:00, 17.00 Samples/s]


In [16]:
# Augmentor genenrates images into ./images/output in following formats:
# - original image: images_original_rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png
# - masked image  : _groundtruth_(1)_images_rock-8.png_325b4fc9-cf0b-4c6a-9574-dc9cc13da6b3.png
# But, https://github.com/rishizek/tensorflow-deeplab-v3 expects:
# - orignal images to be in JPEG format: rock-8.jpg
# - masked images to be in PNG format  : rock-8.png
# - and both of these images to have same filename

output_dir = os.path.join(image_dir, 'output')
for filename in tqdm(os.listdir(output_dir)):
    image_full_path = os.path.join(output_dir, filename)

    if re.match(r"^_groundtruth_", filename):
        new_filename = re.sub(
            r"^_groundtruth_\(1\)_images_",
            '',
            filename
        )
        new_image_full_path= os.path.join(output_dir, new_filename)
        # print('---------')
        # print(new_image_full_path)
        # print('---------')
        os.rename(image_full_path, new_image_full_path)
    else:
        new_filename = re.sub(
            r"^images_original_",
            '',
            filename
        )
        new_filename = re.sub(r".png$", '.jpg', new_filename)
        new_image_full_path= os.path.join(output_dir, new_filename)
        # print(new_image_full_path)
        
        image = Image.open(image_full_path)
        rgb_image = image.convert('RGB')
        rgb_image.save(new_image_full_path)
        
        os.remove(image_full_path)


100%|██████████| 2894/2894 [00:42<00:00, 68.54it/s]
